# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [1]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time

Setting path to train and test data.

In [2]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32
BATCH_SIZE = 128

Loading train data.

In [3]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [4]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [5]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [6]:
train_labels, train_images = load_train_data(TRAIN_DATA_DIR)
val_labels, val_images = load_test_data(TEST_DATA_DIR)

Creating bar chart for dataset Visualization.

In [ ]:
data = {x:train_labels.count(x) for x in train_labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel('Number of examples')
plt.title('Training dataset')
plt.show()

In [76]:
def conv2d(input_layer, filters, padding):
    return tf.layers.conv2d(
        inputs=input_layer,
        filters=filters,
        kernel_size=(5, 5),
        padding=padding,
        activation=tf.nn.elu)

def pooling(input_layer):
    return tf.layers.max_pooling2d(
        inputs=input_layer, 
        pool_size=(2, 2), 
        strides=(2, 2),
        padding='same')

with tf.device('/device:GPU:0'):    
    images = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3], name = "images")
    labels = tf.placeholder(tf.int64, [None])
    
    conv1 = conv2d(images, 12, 'valid')
    pool1 = pooling(conv1)
    
    dropout1 = tf.layers.dropout(inputs=pool1, rate=0.9)
    conv2 = conv2d(dropout1, 24, 'valid')
    pool2 = pooling(conv2)
    
    dropout2 = tf.layers.dropout(inputs=pool2, rate=0.7)
    #conv3 = conv2d(dropout2, 32, 'same')
    #pool3 = pooling(conv3)
    #dropout3 = tf.layers.dropout(inputs=pool3, rate=0.6)
   
    flat = tf.layers.flatten(dropout2)
    f1 = tf.layers.dense(flat, 300, tf.nn.elu)
    
    #dropout = tf.layers.dropout(inputs=flat, rate=0.5)
    
    logits = tf.layers.dense(f1, len(set(train_labels)), tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1, name="prediction")
   
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.global_variables_initializer()

In [77]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [9]:
train_labels_arr = np.array(train_labels)
train_images_arr = np.array(train_images)

In [10]:
def validation(train_images, train_labels, val_images, val_labels):
    train_acc = session.run(accuracy,
                    {images: train_images,
                    labels: train_labels})
    val_acc = session.run(accuracy,
                    {images: val_images,
                    labels: val_labels})
    
    return train_acc, val_acc

In [43]:
def training_log(steps, loss, train_acc, val_acc, training_time):
    print('Steps: {}'.format(steps))
    print('Loss: {0} Training accuracy: {1:.2f}% Validation accuracy: {2:.2f}%'
          .format(loss, train_acc * 100, val_acc * 100))
    minutes, seconds = divmod(training_time, 60)
    hours, minutes = divmod(minutes, 60)
    print('Traning time: {0:02.0f}:{1:02.0f}:{2:05.2f}\n'.format(hours, minutes, seconds))  

In [78]:
def train_model(training_images, training_labels, val_images, val_labels, steps):
    t_start = time.time()
    prev_val_acc = 0
    for i in range(steps + 1):
        indexes = np.random.choice(np.arange(len(train_images)), BATCH_SIZE, replace = False)
        batch_images = train_images_arr[indexes]
        batch_labels = train_labels_arr[indexes]
        loss_value = session.run([optimizer, loss],
                                 {images: batch_images,
                                 labels: batch_labels})[1]
        if (i % 50 == 0):
            training_acc, val_acc = validation(batch_images, batch_labels, val_images, val_labels)
            training_log(i, loss_value, training_acc, val_acc, time.time() - t_start)
            
            if (prev_val_acc < val_acc and i % 500 == 0):
                prev_val_acc = val_acc
                saver.save(session, './model/model')
        
                
train_model(train_images, train_labels, val_images, val_labels, 10000)

Steps: 0
Loss: 3.78278660774 Training accuracy: 8.59% Validation accuracy: 5.80%
Traning time: 00:00:02.26

Steps: 50
Loss: 1.64140045643 Training accuracy: 58.59% Validation accuracy: 55.15%
Traning time: 00:00:07.49

Steps: 100
Loss: 0.951411306858 Training accuracy: 75.00% Validation accuracy: 73.82%
Traning time: 00:00:11.23

Steps: 150
Loss: 0.607889890671 Training accuracy: 85.16% Validation accuracy: 79.41%
Traning time: 00:00:14.94

Steps: 200
Loss: 0.246353298426 Training accuracy: 96.88% Validation accuracy: 83.71%
Traning time: 00:00:18.62

Steps: 250
Loss: 0.234364062548 Training accuracy: 92.97% Validation accuracy: 85.42%
Traning time: 00:00:22.57

Steps: 300
Loss: 0.192865610123 Training accuracy: 95.31% Validation accuracy: 87.82%
Traning time: 00:00:26.43

Steps: 350
Loss: 0.100008904934 Training accuracy: 97.66% Validation accuracy: 88.37%
Traning time: 00:00:30.28

Steps: 400
Loss: 0.161131337285 Training accuracy: 96.88% Validation accuracy: 88.77%
Traning time: 00:

Steps: 3600
Loss: 0.00732702855021 Training accuracy: 100.00% Validation accuracy: 92.56%
Traning time: 00:04:45.57

Steps: 3650
Loss: 0.00201935740188 Training accuracy: 100.00% Validation accuracy: 92.59%
Traning time: 00:04:49.25

Steps: 3700
Loss: 0.00366500904784 Training accuracy: 100.00% Validation accuracy: 93.13%
Traning time: 00:04:52.93

Steps: 3750
Loss: 0.0214396715164 Training accuracy: 100.00% Validation accuracy: 92.38%
Traning time: 00:04:56.62

Steps: 3800
Loss: 0.00129445816856 Training accuracy: 100.00% Validation accuracy: 92.89%
Traning time: 00:05:00.42

Steps: 3850
Loss: 0.00642321724445 Training accuracy: 100.00% Validation accuracy: 92.54%
Traning time: 00:05:04.14

Steps: 3900
Loss: 0.00228062528186 Training accuracy: 100.00% Validation accuracy: 92.52%
Traning time: 00:05:07.82

Steps: 3950
Loss: 0.00285676983185 Training accuracy: 100.00% Validation accuracy: 93.23%
Traning time: 00:05:11.60

Steps: 4000
Loss: 0.0302137359977 Training accuracy: 99.22% Valid

Steps: 7150
Loss: 0.000959967495874 Training accuracy: 100.00% Validation accuracy: 94.22%
Traning time: 00:09:22.44

Steps: 7200
Loss: 0.000105146507849 Training accuracy: 100.00% Validation accuracy: 94.24%
Traning time: 00:09:26.24

Steps: 7250
Loss: 5.41861809324e-05 Training accuracy: 100.00% Validation accuracy: 94.23%
Traning time: 00:09:29.94

Steps: 7300
Loss: 0.000136273432872 Training accuracy: 100.00% Validation accuracy: 94.28%
Traning time: 00:09:33.67

Steps: 7350
Loss: 0.000162819909747 Training accuracy: 100.00% Validation accuracy: 94.24%
Traning time: 00:09:37.61

Steps: 7400
Loss: 0.000287594186375 Training accuracy: 100.00% Validation accuracy: 94.01%
Traning time: 00:09:41.30

Steps: 7450
Loss: 2.76672471955e-05 Training accuracy: 100.00% Validation accuracy: 94.03%
Traning time: 00:09:45.04

Steps: 7500
Loss: 0.000100249460957 Training accuracy: 100.00% Validation accuracy: 94.36%
Traning time: 00:09:48.70

Steps: 7550
Loss: 0.00015074617113 Training accuracy: 10

In [73]:
session.close()